In [1]:
%load_ext autoreload

In [ ]:
data_load.data.columns.any() in ['date', 'region', 'iso3', 'Lat', 'Lon', 'deaths', 'cases', 'recovered']


In [2]:
%autoreload 2
from application.data.data_loader import DataLoader
from configparser import ConfigParser
import pandas as pd
parser = ConfigParser()
parser.read("settings.ini")
data_load = DataLoader(parser)
data_load.load_data()
data_load.latest_load

datetime.datetime(2020, 5, 8, 11, 47, 20, 157348)

In [4]:
len(data_load.countries)

2

In [20]:
class A():
    def foo(self):
        try:
            self.testb()
        except:
            print("not")

class B(A):
    def testb(self):
        print('lol, it works')
b = A()
b.foo()

not


In [36]:
# domain#
class Country:
    def __init__(self, Rank,name,pop2019,pop2018,GrowthRate,area,Density):
        self.Rank=Rank
        self.name=name
        self.pop2019=pop2019
        self.pop2018=pop2018
        self.GrowthRate=GrowthRate
        self.area=area
        self.Density=Density

    def to_dict(self):
        return {
            "Rank":self.Rank,
            "name":self.name,
            "pop2019":self.pop2019,
            "pop2018":self.pop2018,
            "GrowthRate":self.GrowthRate,
            "area":self.area,
            "Density":self.Density

        }




In [37]:
country = Country(1,"test", 12,13,0.32,1234123,22)
country.area

1234123

In [38]:
class Loader(Country):
    def __init__(self, Rank,name,pop2019,pop2018,GrowthRate,area,Density):
        super().__init__(Rank,name,pop2019,pop2018,GrowthRate,area,Density)

    def load_data(self):
        

SyntaxError: unexpected EOF while parsing (<ipython-input-38-7124c446d206>, line 6)

In [47]:
data = pd.read_csv("test.csv", index_col=0)

countries = [Country(**kwargs) for kwargs in data.to_dict(orient='records')]
countries[0].area

9706961.0

In [33]:
countries = Loader(1,"test", 12,13,0.32,1234123,22)
countries.area

1234123

In [24]:
class Parent:
    def __init__(self, txt):
        self.message = txt

    def printmessage(self):
        print(self.message)

class Child(Parent):
    def __init__(self, txt):
        super().__init__(txt)

x = Child("Hello, and welcome!")

x.printmessage()

Hello, and welcome!


In [ ]:
data_load = DataLoader(parser)
data_load.load_data()
data_load.latest_load

In [ ]:
from datetime import datetime
import pandas as pd
from configparser import ConfigParser
from urllib.request import urlopen
import requests
import json
from bs4 import BeautifulSoup    

In [ ]:
class DataLoader:
    def __init__(self, parser: ConfigParser):
        
        self.latest_load = None
        self.data = None
    def load_data(self):
        # local, external

        def load_jhu():
            error_msg = "cannot load jhu data, no url provided"
            try:
                lookup_table = pd.read_csv(parser.get("urls", "jhu_lookup_url"))
                lookup_table.rename(
                    columns={"Country_Region": "region", "Long_": "Lon"}, inplace=True
                )
                return lookup_table
            except:
                print(error_msg)
                return None
            
            if lookup_table:
                def read_prepare_data(url):
                    try:
                        data_raw = pd.read_csv(parser.get("urlss", url))
                        data_raw.rename(columns={"Country/Region": "region"}, inplace=True)
                        data = (
                            data_raw.groupby("region")
                            .sum()
                            .drop(columns=["Lat", "Long"])
                            .reset_index()
                        )

                        return data
                    except:
                        print(error_msg)
                        return None
                
                def create_timeseries(data, lookup_table, value_name):
                    try:
                        
                        id_vars = "region"
                        var_name = "date"
                        timeseries = pd.melt(
                            data, id_vars=id_vars, var_name=var_name, value_name=value_name
                        )
                        timeseries = pd.merge(
                            lookup_table[["iso2", "iso3", "code3", "Lat", "Lon", id_vars]]
                            .groupby(id_vars)
                            .first(),
                            timeseries,
                            on=id_vars,
                            how="inner",
                        )
                        timeseries.loc[:, var_name] = pd.to_datetime(timeseries.loc[:, var_name])
                        return timeseries
                    except:
                        print(error_msg)
                        return None
                        

                confirmed_data = read_prepare_data("jhu_confirmed_url")
                deaths_data = read_prepare_data("jhu_deaths_url")
                recovered_data = read_prepare_data("jhu_recovered_url")

                confirmed = create_timeseries(confirmed_data, lookup, "confirmed")
                deaths = create_timeseries(deaths_data, lookup, "deaths")
                recovered = create_timeseries(recovered_data, lookup, "recovered")

                data = pd.merge(
                    deaths[["date", "region", "iso3", "Lat", "Lon", "deaths"]],
                    confirmed[["date", "confirmed", "iso3"]],
                    on=["iso3", "date"],
                    how="inner",
                )
                data = pd.merge(
                    data,
                    recovered[["date", "recovered", "iso3"]],
                    on=["iso3", "date"],
                    how="inner",
                )

                data.rename(columns={"confirmed": "cases"}, inplace=True)

                return data
                
            else:
                print(error)
                return None
            
                
                
        self.latest_load = datetime.now()
        self.data = load_jhu()
    def prepare(self):
        # join tables, create timeseries
        pass
    def write_data(self):
        # to csv
        if self.data is not None:
            self.data.to_csv("test.csv")
        else:
            print("nothing to write")

In [ ]:
parser = ConfigParser()
parser.read("settings.ini")
parser


In [ ]:
parser.get("urls", "jhu_lookup_url")

In [ ]:
url = "https://raw.githubusercontent.com/ditori1976/covid_app/master/data/countries.csv"
load = Loader(url)
load.load_data()
print(load.latest_load)
print(load.data.head())
load.write_data()

In [ ]:
load = Loader()
load.load_data()
load.write_data()

In [ ]:
class Data(Loader):
    def __init__(self):
        super().__init__(self)

In [ ]:
data = Data()

print(data.latest_load) # latest_load not callable!

data.__dict__#.keys()
